<a href="https://colab.research.google.com/github/Satyamaadi/python/blob/master/Word2VecClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
   

--2022-06-22 16:40:10--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Sentiment%20and%20Emotion%20in%20Text/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2479133 (2.4M) [text/plain]
Saving to: ‘DATAPATH/train_data.csv’

train_data.csv      100%[===================>]   2.36M  --.-KB/s    in 0.05s   

2022-06-22 16:40:11 (49.7 MB/s) - ‘DATAPATH/train_data.csv’ saved [2479133/2479133]



In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('DATAPATH/train_data.csv')

In [4]:
df.columns

Index(['sentiment', 'content'], dtype='object')

In [5]:
df.content[0],df.sentiment[0]

('@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[',
 'empty')

In [6]:
df.sentiment.value_counts()

worry         7433
neutral       6340
sadness       4828
happiness     2986
love          2068
surprise      1613
hate          1187
fun           1088
relief        1021
empty          659
enthusiasm     522
boredom        157
anger           98
Name: sentiment, dtype: int64

In [7]:
df[df['sentiment'] == 'anger']

,sentiment,content
494,anger,fuckin'm transtelecom
527,anger,Working But it's Fridaaaayyyyy
612,anger,Packing I don't like it..
1377,anger,I tried to dye my hair and all i got was a blo...
1384,anger,&quot;locked up abroad&quot; makes bein half b...
...,...,...
25055,anger,@olafsearson Lol - I could try! Seriously tho...
27174,anger,@taylorswift13 jet lag must be soo annoyingg! ...
27246,anger,knows its all going to sort itself out
29917,anger,omg! goooood ass nappy nap jusss woke up bout...


In [8]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

In [9]:
nltk.download('stopwords')
tt = TweetTokenizer(strip_handles=True,preserve_case=False)
mystopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def preprocess_corpus(texts):
    def remove_stops_digits(tokens):
        #Nested function that removes stopwords and digits from a list of tokens
        return [token for token in tokens if token not in mystopwords and not token.isdigit()]
    #This return statement below uses the above function to process twitter tokenizer output further. 
    return [remove_stops_digits(tt.tokenize(content)) for content in texts]

In [11]:
mydata = preprocess_corpus(df['content'])
mycats = df['sentiment']
print(len(mydata), len(mycats))

30000 30000


In [14]:
from sklearn.model_selection import train_test_split
train_data , test_data , train_cats,test_cats = train_test_split(mydata,mycats,random_state=87)

In [16]:
from gensim.models.doc2vec import Doc2Vec , TaggedDocument
train_doc2vec = [TaggedDocument((d), tags=[str(i)]) for i, d in enumerate(train_data)]

In [17]:
model = Doc2Vec(vector_size=50, alpha=0.025, min_count=5, dm =1, epochs=100)
model.build_vocab(train_doc2vec)
model.train(train_doc2vec, total_examples=model.corpus_count, epochs=model.epochs)

In [18]:
train_vectors =  [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in train_data]
test_vectors = [model.infer_vector(list_of_tokens, steps=50) for list_of_tokens in test_data]

In [19]:
test_vectors[0]

array([ 0.00097627,  0.00430379,  0.00205527,  0.00089766, -0.0015269 ,
        0.00291788, -0.00124826,  0.00783546,  0.00927326, -0.00233117,
        0.0058345 ,  0.0005779 ,  0.00136089,  0.00851193, -0.00857928,
       -0.00825741, -0.00959563,  0.0066524 ,  0.00556313,  0.00740024,
        0.00957237,  0.00598317, -0.00077041,  0.00561058, -0.00763451,
        0.00279842, -0.00713293,  0.00889338,  0.00043697, -0.00170676,
       -0.00470889,  0.00548467, -0.00087699,  0.00136868, -0.0096242 ,
        0.00235271,  0.00224191,  0.00233868,  0.00887496,  0.00363641,
       -0.00280984, -0.00125936,  0.00395262, -0.00879549,  0.00333533,
        0.00341276, -0.00579235, -0.00742147, -0.00369143, -0.00272578],
      dtype=float32)